# 🧑‍💻 QA-T Agent: Quality Automation Testing Agent (API Demo)

## Description
This notebook demonstrates how to interact with the QA-T Agent backend API for generating test plans and test cases using AI. You can send requirements or files to your running backend and receive generated test artifacts.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/PraisonAI_QA_Automation_Copilot.ipynb)


# Dependencies

In [ ]:
!pip install praisonaiagents duckduckgo-search

# Set API Key

In [18]:
import os
os.environ["OPENAI_API_KEY"] = "Enter your api key here"  # Replace with your actual key

# Tools Used

In [19]:
from praisonaiagents import Agent
from typing import Dict

# --- PROMPT LOGIC FROM YOUR BACKEND ---

def gherkin_prompt(requirement, feature_name="Test Feature"):
    return f"""
Generate a Gherkin feature file for the following requirement:
{requirement}

Include:
- Feature name: {feature_name}
- Multiple scenarios
- Given-When-Then steps
- Background if needed

Format as:
Feature: {feature_name}
[Description]

Scenario: [Scenario Name]
Given [precondition]
When [action]
Then [expected result]
"""

def selenium_prompt(requirement, test_name="test_case", language="python"):
    return f"""
Generate a Selenium test script in {language} for:
{requirement}

Include:
- Test name: {test_name}
- WebDriver setup
- Page object pattern
- Assertions
- Error handling
"""

def manual_test_prompt(user_story):
    return f"""
Generate manual test cases for this user story:
{user_story}

Include:
- Test case ID
- Test steps
- Expected results
- Test data
- Priority levels

Format as:
Test Case ID: TC001
Test Steps:
1. [step]
2. [step]
Expected Result: [result]
Priority: [High/Medium/Low]
"""

def chat_prompt(question):
    return f"""
Answer this testing question: {question}

Provide:
- Clear explanation
- Best practices
- Examples if relevant
"""

# YAML Prompt

In [20]:
yaml_prompt = """
name: QA Test Generation Agent
description: AI-powered test automation system

tools:
  - name: generate_gherkin
    description: Generate Gherkin feature files from user stories
    parameters:
      requirement: str
      feature_name: str

  - name: generate_selenium
    description: Generate Selenium test scripts in Python
    parameters:
      requirement: str
      test_name: str
      language: str

  - name: generate_manual_tests
    description: Generate manual test cases
    parameters:
      user_story: str

  - name: chat_assistance
    description: Provide testing guidance and explanations
    parameters:
      question: str
"""
print(yaml_prompt)


name: QA Test Generation Agent
description: AI-powered test automation system

tools:
  - name: generate_gherkin
    description: Generate Gherkin feature files from user stories
    parameters:
      requirement: str
      feature_name: str

  - name: generate_selenium
    description: Generate Selenium test scripts in Python
    parameters:
      requirement: str
      test_name: str
      language: str

  - name: generate_manual_tests
    description: Generate manual test cases
    parameters:
      user_story: str

  - name: chat_assistance
    description: Provide testing guidance and explanations
    parameters:
      question: str



# Main (Agent Routing Logic from Your Backend)

In [21]:
class QATestAgent:
    def __init__(self, llm_agent):
        self.llm_agent = llm_agent

    def route_request(self, request: Dict) -> str:
        agent_type = request.get("agentType", "chat").lower()
        requirement = request.get("requirement", "")
        feature_name = request.get("featureName", "Test Feature")
        test_name = request.get("testName", "test_case")
        language = request.get("language", "python")
        user_story = request.get("user_story", requirement)
        question = request.get("question", requirement)

        if agent_type in ["gherkin", "test_generator"]:
            prompt = gherkin_prompt(requirement, feature_name)
        elif agent_type in ["selenium", "selenium_generator"]:
            prompt = selenium_prompt(requirement, test_name, language)
        elif agent_type in ["manual", "manual_testcases"]:
            prompt = manual_test_prompt(user_story)
        else:
            prompt = chat_prompt(question)

        # Use PraisonAI LLM to generate the response
        return self.llm_agent.start(prompt)

# Create the LLM agent (PraisonAI)
llm_agent = Agent(
    instructions="You are a QA Test Generation Agent. Use the prompt to generate the required output.",
    tools=[]
)

qa_agent = QATestAgent(llm_agent)

# Usage Examples

In [22]:
# Example 1: Generate Gherkin
gherkin_result = qa_agent.route_request({
    "agentType": "gherkin",
    "requirement": "User login functionality with email and password",
    "featureName": "User Authentication"
})
print("=== Gherkin Generation ===\n", gherkin_result)

# Example 2: Generate Selenium
selenium_result = qa_agent.route_request({
    "agentType": "selenium",
    "requirement": "Test login page with valid credentials",
    "testName": "test_login_valid",
    "language": "python"
})
print("\n=== Selenium Generation ===\n", selenium_result)

# Example 3: Generate Manual Tests
manual_result = qa_agent.route_request({
    "agentType": "manual",
    "requirement": "As a user, I want to login to the application so that I can access my account"
})
print("\n=== Manual Test Generation ===\n", manual_result)

# Example 4: Chat Assistance
chat_result = qa_agent.route_request({
    "agentType": "chat",
    "requirement": "What is the difference between unit testing and integration testing?"
})
print("\n=== Chat Assistance ===\n", chat_result)

╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: Agent                                                                                                │
│  Role: Assistant                                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Response generated in 8.6s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ Generate a Gherkin feature file for the following requirement: User login functionality with email and password │
│                                                                                                                 │
│ Include:                                                                                                        │
│                                                                                                                 │
│  • Feature name: User Authentication                                                                            │
│  • Multiple scenarios                                                                                           │
│  • Given-When-Then steps                                                                                        │
│  • Background if needed                                                                                         │
│                                                                                                                 │
│ Format as: Feature: User Authentication [Description]                                                           │
│                                                                                                                 │
│ Scenario: [Scenario Name] Given [precondition] When [action] Then [expected result]                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Feature: User Authentication                                                                                   │
│    This feature allows users to log in to the application using their email and password.                       │
│    Successful login grants access to user-specific features and data.                                           │
│                                                                                                                 │
│    Background:                                                                                                  │
│      Given the user is on the login page                                                                        │
│                                                                                                                 │
│    Scenario: Successful login with valid credentials                                                            │
│      Given the user has a registered account with email "user@example.com" and password "password123"           │
│      When the user enters the email "user@example.com"                                                          │
│      And the user enters the password "password123[38;2;166;226;46;48;2;39;40

...

Output()

Response generated in 11.6s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ Answer this testing question: What is the difference between unit testing and integration testing?              │
│                                                                                                                 │
│ Provide:                                                                                                        │
│                                                                                                                 │
│  • Clear explanation                                                                                            │
│  • Best practices                                                                                               │
│  • Examples if relevant                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ Explanation:                                                                                                    │
│                                                                                                                 │
│ Unit Testing:                                                                                                   │
│                                                                                                                 │
│  • Unit testing involves testing individual components or functions of a software application in isolation. The │
│    primary goal is to validate that each unit of the software performs as expected.                             │
│  • It is typically performed by developers during the development phase.                                        │
│  • Unit tests are usually automated and written using testing frameworks such as JUnit for Java, NUnit for      │
│    .NET, or PyTest for Python.                                                                                  │
│  • These tests focus on a single "unit" of code, which could be a function, method, or class, and ensure that   │
│    it behaves correctly under various conditions.                                                               │
│                                                                                                                 │
│ Integration Testing:                                                                                            │
│                                                                                                                 │
│  • Integration testing focuses on testing the interactions between different components or modules of a         │
│    software application.                                                                                        │
│  • The goal is to identify issues that occur when units are combined and to ensure that integrated components   │
│    work together as intended.                                                                                   │
│  • It is usually performed after unit testing and can be done by developers or dedicated testers.               │
│  • Integration tests can be automated or manual and often involve testing interfaces, data flow, and            │
│    communication between modules.                                                                               │
│                                                                                                                 │
│ Best Practices:                                                                                                 │
│                                                                                                                 │
│ Unit Testing Best Practices:                                                                                    │
│                                                                                                                 │
│  • Write tests for all critical paths and edge cases.                                                           │
│  • Keep tests small and focused on a single functionality.                                                      │
│  • Use mock objects to isolate the unit being tested from its dependencies.                                     │
│  • Ensure tests are fast to execute to encourage frequent runs.                                                 │
│  • Maintain clear and descriptive test names to indicate their purpose.                                         │
│                                                                                                                 │
│ Integration Testing Best Practices:                                                                             │
│                                                       


=== Chat Assistance ===
 **Explanation:**

**Unit Testing:**
- Unit testing involves testing individual components or functions of a software application in isolation. The primary goal is to validate that each unit of the software performs as expected.
- It is typically performed by developers during the development phase.
- Unit tests are usually automated and written using testing frameworks such as JUnit for Java, NUnit for .NET, or PyTest for Python.
- These tests focus on a single "unit" of code, which could be a function, method, or class, and ensure that it behaves correctly under various conditions.

**Integration Testing:**
- Integration testing focuses on testing the interactions between different components or modules of a software application.
- The goal is to identify issues that occur when units are combined and to ensure that integrated components work together as intended.
- It is usually performed after unit testing and can be done by developers or dedicated testers.
